In [2]:
import torch
import wandb
import pprint
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy as dpcp
import pandas as pd 
import wandb
import os
import itertools
os.environ["http_proxy"]="http://proxy62.iitd.ac.in:3128"
os.environ["https_proxy"]="http://proxy62.iitd.ac.in:3128"
if torch.cuda.is_available():
  device = torch.device('cuda', index = 0)
else:
  device = 'cpu'

In [3]:
def get_cross_table(versions, models, datasets, metric):
  api = wandb.Api()
  runs = api.runs("fb-graph-proj/fb-graph-proj-dyglib")
  data_names = [dat.split("-")[1] for dat in datasets]
  return_runs = []
  df = pd.DataFrame({f'{model}-{ver}': [torch.nan for _ in datasets] for ver, model in zip(versions,models)}, index = data_names)
  for (ver, model), dat in itertools.product(zip(versions, models), datasets):
    for run in runs:
      # print(run.config['start_time'])
      if run.state == 'finished' and run.config['model'] == model and dat == run.config['dataset'] and ver == run.config['run_name'] and metric in run.summary:
          df.loc[dat.split("-")[1], f'{model}-{ver}'] = round(100 * run.summary[metric], 2)
          break
  return df

In [4]:
# versions = ['old', 'old', 'old', 'mxmdcorr', 'mxmdraw', 'mxmd-normal']
# models = ['TGAT', 'TGN' , 'DyGFormer', 'TGN', 'TGN', 'TGN']
versions = ['old', 'old', 'old', 'old', 'mxmd-v3', 'mxmd-linear']
models = ['TGAT', 'JODIE', 'TGN', 'DyGFormer',  'TGN', 'TGN']
datasets = ["ia-digg-reply", "ia-escorts-dynamic", "ia-retweet-pol", "ia-reality-call", "ia-slashdot-reply-dir", "ia-movielens-user2tags-10m"]
df = get_cross_table(versions, models, datasets, 'test average_precision')

In [5]:
df

,TGAT-old,JODIE-old,TGN-old,DyGFormer-old,TGN-mxmd-v3,TGN-mxmd-linear
digg,77.92,74.50,81.90,77.49,84.25,83.46
escorts,NaN,91.63,90.28,86.57,88.52,89.29
retweet,84.72,84.07,90.74,88.86,85.88,85.51
reality,90.50,95.39,97.85,99.34,97.57,97.49
slashdot,89.88,78.47,92.46,93.07,94.02,93.78
movielens,NaN,85.44,91.29,92.98,88.26,87.61


In [10]:
import pandas as pd

def df_to_latex_table(df, caption="Your Caption Here", label="tab:your_label"):
    # Start the table
    latex_table = "\\begin{table}[h]\n\\centering\n\\begin{tabular}{|l|" + "c|"*len(df.columns) + "}\n\\hline\n"
    
    # Add header
    header = " & ".join(["\\textbf{" + col + "}" for col in df.columns])
    latex_table += "\\textbf{Dataset} & " + header + " \\\\ \\hline\n"
    
    # Add data rows
    for index, row in df.iterrows():
        row_data = " & ".join([f"{x:.2f}" if pd.notnull(x) else "---" for x in row])
        latex_table += f"{index} & {row_data} \\\\ \n"
    
    # Close the table
    latex_table += "\\end{tabular}\n"
    latex_table += f"\\caption{{{caption}}}\n"
    latex_table += f"\\label{{{label}}}\n"
    latex_table += "\\end{table}"
    
    return latex_table


# Convert to LaTeX table
latex_table = df_to_latex_table(df, caption="Model Performance Comparison", label="tab:model_performance")

# Print the LaTeX table
print(latex_table)


\begin{table}[h]
\centering
\begin{tabular}{|l|c|c|c|c|c|c|}
\hline
\textbf{Dataset} & \textbf{TGAT-old} & \textbf{JODIE-old} & \textbf{TGN-old} & \textbf{DyGFormer-old} & \textbf{TGN-mxmd-v3} & \textbf{TGN-mxmd-attn} \\ \hline
digg & 77.92 & 74.50 & 81.90 & 77.49 & 84.25 & 84.04 \\ 
escorts & --- & 91.63 & 90.28 & 86.57 & 88.52 & 88.95 \\ 
retweet & 84.72 & 84.07 & 90.74 & 88.86 & 85.88 & 86.17 \\ 
reality & 90.50 & 95.39 & 97.85 & 99.34 & 97.57 & 97.70 \\ 
slashdot & 89.88 & 78.47 & 92.46 & 93.07 & 94.02 & 93.64 \\ 
movielens & --- & 85.44 & 91.29 & 92.98 & 88.26 & 88.24 \\ 
\end{tabular}
\caption{Model Performance Comparison}
\label{tab:model_performance}
\end{table}


In [40]:
df1 = get_cross_table(versions, models, datasets, 'new node test average_precision')
df1

,TGAT-old,TGN-old,DyGFormer-old,TGN-mxmdcorr,TGN-mxmdraw,TGN-mxmd-normal
digg,65.67,67.21,65.52,64.09,68.35,NaN
escorts,NaN,74.03,76.18,NaN,NaN,67.85
retweet,68.46,76.50,75.87,74.05,75.66,69.10
reality,86.78,95.22,97.73,94.92,94.93,NaN
slashdot,83.87,87.50,88.72,86.24,87.43,NaN
movielens,NaN,81.89,84.98,NaN,NaN,77.03


In [59]:
df2 = get_cross_table(versions, models, datasets, 'new node test first_1_average_precision')
df2

,TGAT-old,TGN-old,DyGFormer-old,TGN-mxmdcorr,TGN-mxmdraw,TGN-mxmd-normal
digg,63.21,60.26,63.23,66.43,62.34,NaN
escorts,NaN,61.53,62.08,NaN,NaN,61.91
retweet,53.30,68.50,58.19,64.95,67.71,53.50
reality,74.17,89.28,91.19,83.85,84.49,NaN
slashdot,85.67,89.46,86.83,87.06,88.20,NaN
movielens,NaN,66.38,65.54,NaN,NaN,56.87


In [60]:
df2 = get_cross_table(versions, models, datasets, 'new node test first_3_average_precision')
df2

,TGAT-old,TGN-old,DyGFormer-old,TGN-mxmdcorr,TGN-mxmdraw,TGN-mxmd-normal
digg,62.79,62.19,63.11,61.41,63.89,NaN
escorts,NaN,65.71,66.09,NaN,NaN,60.51
retweet,56.40,66.97,63.15,63.93,65.66,57.58
reality,77.29,90.27,93.78,88.69,89.25,NaN
slashdot,81.19,86.40,86.23,84.21,85.18,NaN
movielens,NaN,66.55,69.14,NaN,NaN,63.88


In [44]:
api = wandb.Api()
runs = api.runs("fb-graph-proj/fb-graph-proj-dyglib")

In [56]:
run =runs[0]
run.start_time

AttributeError: '<Run fb-graph-proj/fb-graph-proj-dyglib/hy0wo06a (finished)>' object has no attribute 'start_time'